In [19]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

In [20]:
# Cargar el archivo Excel
df = pd.read_excel('groudon.xlsx')

In [21]:
# Filtrar los registros donde 'Magnitud' no sea "no calculable" y convertir la columna 'Magnitud' a numérico
df['Magnitud'] = pd.to_numeric(df['Magnitud'], errors='coerce')  # Convierte a NaN cualquier valor no numérico
df = df.dropna(subset=['Magnitud'])  # Elimina filas con valores NaN en 'Magnitud'

# Convertir fecha a datetime y extraer año
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
df['Año'] = df['Fecha'].dt.year

# Limpiar nulos importantes
df = df.dropna(subset=['Latitud', 'Longitud', 'Magnitud', 'Profundidad', 'Año'])



In [22]:
# Asignar el tipo de marcador según el rango de años
def asignar_forma_anno(year):
    if year < 1960:
        return 'star'
    elif 1961 <= year <= 1980:
        return 'square'
    elif 1981 <= year <= 2000:
        return 'triangle-down'
    elif 2001 <= year <= 2020:
        return 'triangle-up'
    else:
        return 'circle'

In [23]:
df['Forma'] = df['Año'].apply(asignar_forma_anno)

# Crear sliders
year_slider = widgets.IntRangeSlider(
    value=[df['Año'].min(), df['Año'].max()],
    min=df['Año'].min(),
    max=df['Año'].max(),
    step=1,
    description='Año:',
    continuous_update=False
)

mag_slider = widgets.FloatRangeSlider(
    value=[df['Magnitud'].min(), df['Magnitud'].max()],
    min=df['Magnitud'].min(),
    max=df['Magnitud'].max(),
    step=0.1,
    description='Magnitud:',
    continuous_update=False
)

depth_slider = widgets.FloatRangeSlider(
    value=[df['Profundidad'].min(), df['Profundidad'].max()],
    min=df['Profundidad'].min(),
    max=df['Profundidad'].max(),
    step=1,
    description='Profundidad:',
    continuous_update=False
)




In [24]:
# Función para actualizar el mapa
def actualizar_mapa(rango_ano, rango_mag, rango_prof):
    df_filtrado = df[
        (df['Año'] >= rango_ano[0]) & (df['Año'] <= rango_ano[1]) &
        (df['Magnitud'] >= rango_mag[0]) & (df['Magnitud'] <= rango_mag[1]) &
        (df['Profundidad'] >= rango_prof[0]) & (df['Profundidad'] <= rango_prof[1])
    ]
    
    fig = go.Figure()

    # Añadir trazos para cada tipo de marcador
    for forma in df_filtrado['Forma'].unique():
        df_forma = df_filtrado[df_filtrado['Forma'] == forma]
        fig.add_trace(go.Scattermapbox(
            lat=df_forma['Latitud'],
            lon=df_forma['Longitud'],
            mode='markers',
            marker=dict(
                size=df_forma['Magnitud'] * 5,  # Ajustar tamaño según la magnitud
                color=df_forma['Magnitud'],
                colorscale='inferno',
                symbol=forma,  # Forma según el año
                showscale=True
            ),
            hoverinfo='text',
            hovertext=df_forma['Referencia de localizacion'],
            name=f'Eventos {forma}'
        ))
    
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=3,
        height=600,
        title='Eventos sísmicos filtrados',
    )

    fig.show()

In [ ]:
# Interfaz
ui = widgets.VBox([year_slider, mag_slider, depth_slider])
out = widgets.interactive_output(actualizar_mapa, {
    'rango_ano': year_slider,
    'rango_mag': mag_slider,
    'rango_prof': depth_slider
})

display(ui, out)